# Tridiagonal Matrix Solver algorithm and Test

## M matrix

###         |b[0] c[0] 0      ...   ...     ...     0 |
###         |a[1] b[1] c[1]     ...    ...      0 |
###         | ...    a[2] b[2] c[2] ...0 |
###         | ...    a[n-2] b[n-2] c[n-2] |
###         | ...    ...   ...a[n-1] b[n-1] |

# Import

In [ ]:
import numpy as np

# Define matrices and vectors

In [ ]:
# Define n in our nxn matrix
n=31

# M matrix
diag = 2.*np.ones((n))
l_diag = np.ones((n))
u_diag = np.ones((n))

l_diag[0] = 0.
u_diag[n-1] = 0.

# b vector
# Initialize
#print(np.round(n/2.))
b_vec = np.ones((n))
for i in range(1,n):
    if i <= (np.round(n/2.)): 
        b_vec[i-1] = i
    else:
        b_vec[i-1] = (n+1)-i
print(b_vec)

# Define the functions to solve Mx = b

## The solver

In [ ]:
# Thomas Algorithm if diagonally dominant
# Forward elimination for a tridiagonal matrix
# Based on the algorithm found in Dullemond Ch4 notes
# Backward Substition is then applied
def tri_solver(a,b,c,y,n):
    """
    Input: 
        a = lower diagonals of matrix M
        b = diagonals of matrix M
        c = upper diagonals of matrix M
        y = the solution array in the matrix equation: Mx = y (nx1)
        n = the number of rows in matrix m
    Output:
        A new matrix or vector of S values 
    """
    
    # Check to see if matrix is diagonally dominant
    flag = 0
    comp_arr = c[:-1] + a[1:] #Add the upper and lower diagonals
    dim = len(comp_arr)
    for i in range(0,dim):
        if comp_arr[i]>=c[i+1]: #See if the sum of the upper and lower diagonals are larger than the diagonal element
            flag = 1 # Not diagonally dominant
            break # If there is even one row that isn't diagonally dominant, we can't use the Thomas Algorithm
            
    # The Thomas Algorithm for diagonally dominant matrices
    if flag == 0:
        #-----------------------------------------------------------
        # Perform forward elimination and backward substitution on M
        # Thomas Algorithm: a simplified form of Gaussian Elimination
        # specifically for tridiagonal matrices 
        
        # Define column arrays for necessary row-specific constants
        # This automatically sets gamma[0] = beta[0] = 0 which is 
        # necessary for the problem
        g = np.zeros((n))
        beta = np.zeros((n))
    
        for i in range(0,n-1):
            g[i+1] = (-a[i]/(c[i]*g[i] + b[i]))
            beta[i+1] = (y[i] - beta[i]*c[i])/b[i]
    
        # Use the gamma and beta constant arrays to compute 
        # what is essentially a back substitution to finish the
        # calculation of MX = Y
        X = np.zeros((n+1))
        #X[0:n] = S
        for k in range(1,n+1):
            # Actually need to go from n, n-1, n-2,...,1
            # So create a new place holder that goes backwards
            c = n - k #Count backwards
            X[c] = g[c]*X[c+1] + beta[c]
    
        # Return the new solution to J
        return(X[0:n])
    
    # Non-diagonally dominant matrices
    else:
        # Initialize new arrays
        c_prime = np.zeros((n))
        y_prime = np.zeros((n))
        X = np.zeros((n))
        
        for i in range(0,n):
            if i==0:
                c_prime[i] = c[i]/b[i]
                y_prime[i] = y[i]/b[i]
            elif i>0 and i<n-1:
                c_prime[i] = c[i]/(b[i]-a[i]*c_prime[i-1])
                y_prime[i] = (y[i]-a[i]*y_prime[i-1])/(b[i]-a[i]*c_prime[i-1])
            elif i==n-1:
                y_prime[i] = (y[i]-a[i]*y_prime[i-1])/(b[i]-a[i]*c_prime[i-1])
                X[i] = y_prime[i]
        for i in range(2,n):
            j = n-i
            X[j] = y_prime[j] - c_prime[j]*X[j+1]
        return(X)

# Results

In [ ]:
# Find x
x_arr = tri_solver(l_diag,diag,u_diag,b_vec,n)

In [ ]:
print(x_arr)

## True answer

### x = [0 1 0 2 0 3 0 4 0 5 0 6 0 7 0 8 0 7 0 6 0 5 0 4 0 3 0 2 0 1 0]